In [10]:
# LSTM+CNN+Dropout
# trained on a variety of feature combinations

# First, import necessary libraries
import tensorflow as tf
import numpy as np
import keras.backend as K

In [11]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Conv1D, BatchNormalization
from keras.layers import LSTM
# my custom data_utils file
from data_utils_local08 import *

In [12]:
# uploading the X values from the autoencoder
X_input = np.load('../Desktop/VGG16_feature_data.npy')
print(X_input[1])

# normalize X_input

# for combinations of features
#X_input = np.concatenate((X_input_VAE, X_input_VGG), axis=2)

[[ -4.479     0.6116   -2.4313  ...   0.40712  -1.26     -0.38584]
 [ -4.4253    0.43527  -2.4657  ...   0.40969  -1.3558   -0.44199]
 [ -5.7516   -7.8904   -6.8481  ... -10.082    -9.5151    5.0102 ]
 ...
 [-10.531     1.5749   -2.5768  ...   3.0096   -4.0217    5.9333 ]
 [ -9.3       3.5598   -2.054   ...   4.5326   -5.1147    5.7942 ]
 [ -4.6847    2.7427   -3.7783  ...   1.6169    1.38      3.8095 ]]


In [13]:
# uploading the Y values
y_data_input = fear_oneHot(212, 'fear_annotations_part01/MEDIAEVAL18_7_Fear.txt')
print(y_data_input.shape)

(212,)


In [14]:
# partition data into training and testing sets
train_num = 4
val_num = 3

X_train_data = X_input[:train_num, :, :]
X_valid_data = X_input[train_num:train_num+val_num, :, :]

print(X_train_data.shape)
print(X_valid_data.shape)

(4, 212, 4096)
(3, 212, 4096)


In [15]:
# uploading the y training data
timesteps = X_train_data.shape[1]
data_dim = X_train_data.shape[2]

# set up y_train_data master array
Y_train_data = np.zeros([train_num, timesteps])

# for each movie number between and including 7 and 13
for num in range(train_num):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+num))
    # create a one-hot vector
    y_data = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_train_data[num, :] = y_data
print(Y_train_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [16]:
# upload the Y validation set

Y_valid_data = np.zeros([val_num, timesteps])

# for each movie number in validation set
for num in range(val_num):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+ train_num + num))
    # create a one-hot vector
    y_valid = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_valid_data[num, :] = y_valid
print(Y_valid_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [19]:
# model architecture
model = Sequential()

# normalization layer
#model.add(BatchNormalization(input_shape=(timesteps, data_dim)))

model.add(Conv1D(25, kernel_size=3, activation="sigmoid", input_shape=(timesteps, data_dim)))
# input_shape=(timesteps, data_dim)

model.add(LSTM(timesteps, return_sequences=True))
# input_shape=(timesteps, data_dim)
# dropout layer
model.add(Dropout(0.5))

# another LSTM layer
model.add(LSTM(timesteps, return_sequences=True))

# necessary flatten layer
model.add(Flatten()) 

# add the final dense layer and then softmax
model.add(Dense(timesteps, activation='softmax'))

In [20]:
# a new metric of evaluation! the F-score!
def FScore2(y_true, y_pred):
    '''
    The F score, beta=2
    '''
    B2 = K.variable(4)
    OnePlusB2 = K.variable(5)
    pred = K.round(y_pred)
    tp = K.sum(K.cast(K.less(K.abs(pred - K.clip(y_true, .5, 1.)), 0.01), 'float32'), -1)
    fp = K.sum(K.cast(K.greater(pred - y_true, 0.1), 'float32'), -1)
    fn = K.sum(K.cast(K.less(pred - y_true, -0.1), 'float32'), -1)

    f2 = OnePlusB2 * tp / (OnePlusB2 * tp + B2 * fn + fp)

    return K.mean(f2)

In [21]:
# another attempt at f1 score
def precision(y_true, y_pred):
    #"""Precision metric.
    # Only computes a batch-wise average of precision.
    #Computes the precision, a metric for multi-label classification of
    #how many selected items are relevant.
    #"""
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    #"""Recall metric.
    # Only computes a batch-wise average of recall.
    # Computes the recall, a metric for multi-label classification of
    #how many relevant items are selected.
    #"""
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    #"""Computes the F score.
     #The F score is the weighted harmonic mean of precision and recall.
    #Here it is only computed as a batch-wise average, not globally.
    # This is useful for multi-label classification, where input samples can be
    #classified as sets of labels. By only using accuracy (precision) a model
    #would achieve a perfect score by simply assigning every class to every
    #input. In order to avoid this, a metric should penalize incorrect class
    #assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    #computes this, as a weighted mean of the proportion of correct class
    #assignments vs. the proportion of incorrect class assignments.
    # With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    #correct classes becomes more important, and with beta > 1 the metric is
    #instead weighted towards penalizing incorrect class assignments.
    #"""
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def fmeasure(y_true, y_pred):
    #"""Computes the f-measure, the harmonic mean of precision and recall.
    #Here it is only computed as a batch-wise average, not globally.
    #"""
    return fbeta_score(y_true, y_pred, beta=1)

In [24]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy', FScore2])

In [25]:
# running the LSTM model
model.fit(X_train_data, Y_train_data, epochs = 20, validation_data=(X_valid_data, Y_valid_data))
print("finished training!")

Train on 4 samples, validate on 3 samples
Epoch 1/20
4/4 [==============================] - 2s 625ms/step - loss: 0.8115 - binary_accuracy: 0.8054 - FScore2: 0.0000e+00 - val_loss: 3.1164 - val_binary_accuracy: 0.7689 - val_FScore2: 0.0000e+00
Epoch 2/20
4/4 [==============================] - 1s 218ms/step - loss: 1.6969 - binary_accuracy: 0.8066 - FScore2: 0.1389 - val_loss: 1.4172 - val_binary_accuracy: 0.7689 - val_FScore2: 0.0000e+00
Epoch 3/20
4/4 [==============================] - 1s 353ms/step - loss: 1.1336 - binary_accuracy: 0.8054 - FScore2: 0.0000e+00 - val_loss: 1.7805 - val_binary_accuracy: 0.7689 - val_FScore2: 0.0000e+00
Epoch 4/20
4/4 [==============================] - 1s 344ms/step - loss: 1.1147 - binary_accuracy: 0.8066 - FScore2: 0.1389 - val_loss: 1.4467 - val_binary_accuracy: 0.7689 - val_FScore2: 0.0000e+00
Epoch 5/20
4/4 [==============================] - 1s 300ms/step - loss: 0.9160 - binary_accuracy: 0.8054 - FScore2: 0.0000e+00 - val_loss: 1.5370 - val_binary

In [61]:
# trying to view the model output
out = model.predict(X_train_data)
print("model prediction:")
print(out[2])
print("target:")
print(Y_train_data[0])

print("before 64:")
print(out[0][63])
print("64:")
print(out[0][65])

print("rounded")
print(np.round(out)[0])

model prediction:
[2.38958673e-05 2.63057191e-05 2.37448912e-05 2.68017175e-05
 2.60396464e-05 2.54288461e-05 2.33226492e-05 2.71342287e-05
 2.43517934e-05 3.10983633e-05 2.43608974e-05 2.32270286e-05
 2.27290620e-05 2.73130590e-05 2.68658223e-05 2.47956687e-05
 2.33350183e-05 2.44454714e-05 2.45334722e-05 2.64859809e-05
 2.14069860e-05 2.43648028e-05 2.34919680e-05 2.40291447e-05
 2.40296704e-05 2.44170224e-05 9.06482019e-05 8.71397860e-05
 8.60483851e-05 1.33781679e-04 9.07979702e-05 8.94258337e-05
 9.07842914e-05 8.57495397e-05 8.38751512e-05 1.19831377e-04
 8.37490152e-05 5.97990453e-01 3.45843554e-01 9.91230991e-05
 9.12500618e-05 9.07953727e-05 8.89375660e-05 1.42566263e-04
 1.27050414e-04 9.37161312e-05 8.16072352e-05 9.36284050e-05
 9.39250531e-05 8.68483330e-05 8.91739182e-05 1.41794735e-04
 9.89116845e-04 1.30350981e-03 1.39874732e-03 1.21719681e-03
 1.30413775e-03 1.33993581e-03 1.16660981e-03 1.16246229e-03
 1.64742698e-04 8.43369198e-05 7.85895900e-05 9.31399263e-05
 6.358

In [ ]:
#try visualizing this model at some point?